In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os, shutil, subprocess, urllib.request, urllib
from dateutil import parser
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from time import strptime
from datetime import datetime
from joblib import Parallel, delayed
from tqdm import tqdm, trange

pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 1000)

In [5]:
# Should mp3 sample files be downloaded to the local drive (yes/no)?
download_mp3 = 'yes'

In [2]:
def wait_for_internet_connection():
    while True:
        try:
            response = urllib.request.urlopen('https://facebook.com',timeout=1)
            return
        except urllib.request.URLError:
            pass


# Enter location of Chromedriver
chromedriver = ''
os.environ["webdriver.chrome.driver"] = chromedriver

chromeOptions = Options()
# Block images to speed up the page loading
prefs = {"profile.managed_default_content_settings.images":2}
chromeOptions.add_experimental_option("prefs",prefs)
chromeOptions.add_argument('headless')

appended_data_0 = []

def process_audible(x):

    driver = webdriver.Chrome(chromedriver, chrome_options=chromeOptions)
    wait_for_internet_connection()
    driver.get('https://www.audible.com/search?ref=a_search_c4_pageNum_1&pageSize=50&page=' + str(x))
    src = driver.page_source
    for y in range(1, 51):
        book_link_1 = '#center-3 > div > span > ul > li:nth-child('
        book_link_2 = ') > div > div.bc-col-responsive.bc-spacing-top-none.bc-col-8 > div > div.bc-col-responsive.bc-col-6 > div > div > span > ul > li:nth-child(1) > h3 > a'
        button = driver.find_element_by_css_selector(book_link_1 + str(y) + book_link_2)
        button.click()
        
        book_url = driver.current_url.split('?ref')[0]
        try:
            book_length = driver.find_element_by_css_selector('#center-1 > div > div > div > div.bc-col-responsive.bc-col-5 > span > ul > li.bc-list-item.runtimeLabel').text
            mp3_source = driver.page_source.split('data-mp3="')[1].split('.mp3"')[0]+str('.mp3')
            performance_5_star = int(driver.find_element_by_css_selector('#center-14 > div > div > div > div:nth-child(2) > span > ul > li:nth-child(2) > div > div.bc-col-responsive.bc-col-2').text.replace(',', ''))
            performance_4_star = int(driver.find_element_by_css_selector('#center-14 > div > div > div > div:nth-child(2) > span > ul > li:nth-child(3) > div > div.bc-col-responsive.bc-col-2').text.replace(',', ''))
            performance_3_star = int(driver.find_element_by_css_selector('#center-14 > div > div > div > div:nth-child(2) > span > ul > li:nth-child(4) > div > div.bc-col-responsive.bc-col-2').text.replace(',', ''))
            performance_2_star = int(driver.find_element_by_css_selector('#center-14 > div > div > div > div:nth-child(2) > span > ul > li:nth-child(5) > div > div.bc-col-responsive.bc-col-2').text.replace(',', ''))
            performance_1_star = int(driver.find_element_by_css_selector('#center-14 > div > div > div > div:nth-child(2) > span > ul > li:nth-child(6) > div > div.bc-col-responsive.bc-col-2').text.replace(',', ''))
            num_perf_ratings = performance_5_star + performance_4_star + performance_3_star + performance_2_star + performance_1_star
            
            temp1 = [int(s) for s in book_length.split() if s.isdigit()]
            if len(temp1)==2:
                length_min = temp1[0]*60+temp1[1]
            if len(temp1)==1:
                if 'min' in book_length:
                    length_min = temp1[0]
                if 'hr' in book_length:
                    length_min = temp1[0]*60
            
            if length_min > 0 and num_perf_ratings > 0:
                    
                    if download_mp3.lower() = 'yes':
                        with urllib.request.urlopen(mp3_source) as response, open(str(((x-1)*50)+y)+str('.mp3'), 'wb') as out_file:
                                shutil.copyfileobj(response, out_file)

                    try:    
                        title = driver.find_element_by_css_selector('#center-1 > div > div > div > div.bc-col-responsive.bc-col-5 > span > ul > li:nth-child(1) > h1').text
                    except:
                        title = 'NULL'
                    try:
                        author = driver.find_element_by_css_selector('#center-1 > div > div > div > div.bc-col-responsive.bc-col-5 > span > ul > li.bc-list-item.authorLabel > a').text
                    except:
                        author = 'NULL'
                    try:
                        narrator = driver.find_element_by_css_selector('#center-1 > div > div > div > div.bc-col-responsive.bc-col-5 > span > ul > li.bc-list-item.narratorLabel > a').text
                    except:
                        narrator = 'NULL'
                    try:
                        price = float(driver.find_element_by_css_selector('#adbl-buy-box-price > p > span:nth-child(2)').text.replace('$', ''))
                    except:
                        price = 'NULL'
                    try:
                        language = driver.find_element_by_css_selector('#center-1 > div > div > div > div.bc-col-responsive.bc-col-5 > span > ul > li.bc-list-item.languageLabel').text.split('Language: ')[1]
                    except:
                        language = 'NULL'
                    try:                    
                        release_date = parser.parse(driver.find_element_by_css_selector('#center-1 > div > div > div > div.bc-col-responsive.bc-col-5 > span > ul > li.bc-list-item.releaseDateLabel').text.split('Release date: ')[1]).date()
                    except:
                        release_date = 'NULL'
                    try:
                        category1 = driver.find_element_by_css_selector('#center-1 > div > div > nav > a:nth-child(1)').text
                    except:
                        category1 = 'NULL'
                    try:
                        category2 = driver.find_element_by_css_selector('#center-1 > div > div > nav > a:nth-child(3)').text
                    except:
                        category2 = 'NULL'
                    try:
                        publisher = driver.find_element_by_css_selector('#center-1 > div > div > div > div.bc-col-responsive.bc-col-5 > span > ul > li.bc-list-item.publisherLabel > a').text
                    except:
                        publisher = 'NULL'
                    try:
                        rating_overall = float(driver.find_element_by_css_selector('#center-1 > div > div > div > div.bc-col-responsive.bc-col-5 > span > ul > li.bc-list-item.ratingsLabel > span:nth-child(3)').text)
                    except:
                        rating_overall = 'NULL'
                    try:
                        rating_overall_num = int(driver.find_element_by_css_selector('#center-1 > div > div > div > div.bc-col-responsive.bc-col-5 > span > ul > li.bc-list-item.ratingsLabel > a').text.replace('(','').replace(')', '').split(' ratings')[0].replace(',', ''))               
                    except:
                        rating_overall_num = 'NULL'
                    try:
                        overall_5_star = int(driver.find_element_by_css_selector('#center-14 > div > div > div > div:nth-child(1) > span > ul > li:nth-child(2) > div > div.bc-col-responsive.bc-col-2').text.replace(',', ''))
                    except:
                        overall_5_star = 'NULL'
                    try:
                        overall_4_star = int(driver.find_element_by_css_selector('#center-14 > div > div > div > div:nth-child(1) > span > ul > li:nth-child(3) > div > div.bc-col-responsive.bc-col-2').text.replace(',', ''))
                    except:
                        overall_4_star = 'NULL'
                    try:
                        overall_3_star = int(driver.find_element_by_css_selector('#center-14 > div > div > div > div:nth-child(1) > span > ul > li:nth-child(4) > div > div.bc-col-responsive.bc-col-2').text.replace(',', ''))
                    except:
                        overall_3_star = 'NULL'
                    try:
                        overall_2_star = int(driver.find_element_by_css_selector('#center-14 > div > div > div > div:nth-child(1) > span > ul > li:nth-child(5) > div > div.bc-col-responsive.bc-col-2').text.replace(',', ''))
                    except:
                        overall_2_star = 'NULL'
                    try:
                        overall_1_star = int(driver.find_element_by_css_selector('#center-14 > div > div > div > div:nth-child(1) > span > ul > li:nth-child(6) > div > div.bc-col-responsive.bc-col-2').text.replace(',', ''))
                    except:
                        overall_1_star = 'NULL'
                    try:
                        story_5_star = int(driver.find_element_by_css_selector('#center-14 > div > div > div > div:nth-child(3) > span > ul > li:nth-child(2) > div > div.bc-col-responsive.bc-col-2').text.replace(',', ''))
                    except:
                        story_5_star = 'NULL'
                    try:
                        story_4_star = int(driver.find_element_by_css_selector('#center-14 > div > div > div > div:nth-child(3) > span > ul > li:nth-child(3) > div > div.bc-col-responsive.bc-col-2').text.replace(',', ''))
                    except:
                        story_4_star = 'NULL'
                    try:
                        story_3_star = int(driver.find_element_by_css_selector('#center-14 > div > div > div > div:nth-child(3) > span > ul > li:nth-child(4) > div > div.bc-col-responsive.bc-col-2').text.replace(',', ''))
                    except:
                        story_3_star = 'NULL'
                    try:
                        story_2_star = int(driver.find_element_by_css_selector('#center-14 > div > div > div > div:nth-child(3) > span > ul > li:nth-child(5) > div > div.bc-col-responsive.bc-col-2').text.replace(',', ''))
                    except:
                        story_2_star = 'NULL'
                    try:                        
                        story_1_star = int(driver.find_element_by_css_selector('#center-14 > div > div > div > div:nth-child(3) > span > ul > li:nth-child(6) > div > div.bc-col-responsive.bc-col-2').text.replace(',', ''))
                    except:
                        story_1_star = 'NULL'

                    my_list = {'book_num': str(((x-1)*50)+y), 'title':title,'author':author,'narrator':narrator,'length_min':length_min,'price':price,'language':language,'release_date':release_date,'category1':category1,'category2':category2,'publisher':publisher,'rating_overall':rating_overall,'rating_overall_num':rating_overall_num,'overall_5_star':overall_5_star,'overall_4_star':overall_4_star,'overall_3_star':overall_3_star,'overall_2_star':overall_2_star,'overall_1_star':overall_1_star,'performance_5_star':performance_5_star,'performance_4_star':performance_4_star,'performance_3_star':performance_3_star,'performance_2_star':performance_2_star,'performance_1_star':performance_1_star,'story_5_star':story_5_star,'story_4_star':story_4_star,'story_3_star':story_3_star,'story_2_star':story_2_star,'story_1_star':story_1_star,'book_url':book_url,'mp3_source':mp3_source}
                    appended_data_0.append(my_list)
                    
            else:
                print(title, book_url, length_min, num_perf_ratings)
                pass

        except Exception as e: 
            #print(e, book_url)
            driver.execute_script("window.history.go(-1)")
            continue

        driver.execute_script("window.history.go(-1)")
        time.sleep(random.uniform(2, 4))
    driver.close()
    return appended_data_0

In [3]:
start = time.time()

# This will fetch the first two search reuslt pages (out of ~9,000 pages), with 50 results per page
z = 2
results = Parallel(n_jobs=-2)(delayed(process_audible)(x) for x in tqdm(range(1, z+1)))
results_df = pd.DataFrame([val for sublist in results for val in sublist]).drop_duplicates()

end = time.time()
print(end - start)

100%|██████████| 2/2 [00:00<00:00, 39.29it/s]


193.5641667842865


In [4]:
results_df

,author,book_num,book_url,category1,category2,language,length_min,mp3_source,narrator,overall_1_star,overall_2_star,overall_3_star,overall_4_star,overall_5_star,performance_1_star,performance_2_star,performance_3_star,performance_4_star,performance_5_star,price,publisher,rating_overall,rating_overall_num,release_date,story_1_star,story_2_star,story_3_star,story_4_star,story_5_star,title
0,Stephen King,1,https://www.audible.com/pd/Fiction/The-Outsider-Audiobook/B07957SPGB,Fiction,Horror,English,1119,https://samples.audible.com/bk/sans/008262/bk_sans_008262_sample.mp3,Will Patton,46,64,262,1017,3590,64,58,160,550,3845,34.99,Simon & Schuster Audio,4.6,4979,2018-05-22,50,80,302,969,3247,The Outsider
1,Jordan B. Peterson,2,https://www.audible.com/pd/Science-Technology/12-Rules-for-Life-Audiobook/B0797YBP7N,Science & Technology,Psychology & The Mind,English,940,https://samples.audible.com/bk/prhc/000058/bk_prhc_000058_sample.mp3,Jordan B. Peterson,531,409,639,1645,13529,408,268,622,1522,12340,36.33,Random House Canada,4.6,16753,2018-01-23,521,374,622,1601,11912,12 Rules for Life
2,Dean Koontz,3,https://www.audible.com/pd/Fiction/Frankenstein-Prodigal-Son-Audiobook/B077V4855B,Fiction,Horror,English,543,https://samples.audible.com/bk/brll/009759/bk_brll_009759_sample.mp3,Christopher Lane,1,5,29,68,183,0,4,10,60,187,17.49,Brilliance Audio,4.5,286,2017-12-04,1,11,23,58,167,Frankenstein: Prodigal Son
3,Bill Clinton,4,https://www.audible.com/pd/Mysteries-Thrillers/The-President-Is-Missing-Audiobook/B076HY6P86,Mysteries & Thrillers,Political,English,773,https://samples.audible.com/bk/hach/003529/bk_hach_003529_sample.mp3,Dennis Quaid,10,4,11,25,53,12,4,8,16,58,30.79,Hachette Audio,4.0,103,2018-06-04,9,3,11,21,54,The President Is Missing
4,David Sedaris,5,https://www.audible.com/pd/Comedy/Calypso-Audiobook/B07BGGTGBQ,Comedy,Memoirs,English,398,https://samples.audible.com/bk/hach/004317/bk_hach_004317_sample.mp3,David Sedaris,8,6,13,76,670,3,5,4,30,671,30.79,Hachette Audio,4.8,773,2018-05-29,7,7,10,84,601,Calypso
5,Mark Manson,6,https://www.audible.com/pd/Self-Development/The-Subtle-Art-of-Not-Giving-a-F-ck-Audiobook/B01I28NFEE,Self Development,Motivation & Inspiration,English,317,https://samples.audible.com/bk/harp/005490/bk_harp_005490_sample.mp3,Roger Wayne,1563,1736,5450,16757,54785,919,843,3223,12024,53474,23.95,HarperAudio,4.5,80291,2016-09-13,1629,1820,5647,15205,45791,The Subtle Art of Not Giving a F*ck
6,Catherine Ryan Hyde,7,https://www.audible.com/pd/Fiction/The-Wake-Up-Audiobook/B0773XMNTK,Fiction,Literary,English,542,https://samples.audible.com/bk/brll/009646/bk_brll_009646_sample.mp3,Nick Podehl,7,12,33,91,197,4,9,20,65,194,17.49,Brilliance Audio,4.4,340,2017-12-05,7,6,34,69,177,The Wake Up
7,Rachel Hollis,8,https://www.audible.com/pd/Self-Development/Girl-Wash-Your-Face-Audiobook/B077H1PRL8,Self Development,Motivation & Inspiration,English,424,https://samples.audible.com/bk/tnwd/000792/bk_tnwd_000792_sample.mp3,Rachel Hollis,59,35,98,265,3133,41,26,69,192,2872,21.67,Thomas Nelson,4.8,3590,2018-02-06,60,24,92,248,2759,"Girl, Wash Your Face"
8,Michael Pollan,9,https://www.audible.com/pd/Bios-Memoirs/How-to-Change-Your-Mind-Audiobook/B07B1J873J,Bios & Memoirs,"Artists, Writers, & Musicians",English,814,https://samples.audible.com/bk/peng/003895/bk_peng_003895_sample.mp3,Michael Pollan,5,3,17,66,574,3,3,10,61,532,31.50,Penguin Audio,4.8,665,2018-05-15,5,3,9,68,521,How to Change Your Mind
9,Mel Robbins,10,https://www.audible.com/pd/Business/Kick-Ass-with-Mel-Robbins-Audiobook/B07BZV7FKK,Business,Leadership,English,385,https://samples.audible.com/or/orig/000199/or_orig_000199_sample.mp3,Mel Robbins,59,36,60,160,1728,45,21,37,129,1609,29.95,Audible Studios,4.7,2043,2018-05-08,54,29,62,155,1530,Kick Ass with Mel Robbins
